# Nettoyage des données network

Ce notebook ne contient que la préparation des données, afin d'obtenir les données préparées plus rapidement.  
C'est un copier-coller des parties Nettoyage du notebook `no_run_preparation_network.ipynb`, sans les vérifications.  
Les explications des choix de préparation sont dans l'autre notebook.

In [1]:
import pandas as pd
import os
from pickleshare import PickleShareDB
import numpy as np

## Chargement des données

In [2]:
data_dir = '../prep_data' 
os.makedirs(data_dir, exist_ok=True)
db = PickleShareDB(os.path.join(data_dir, 'kity'))

df_all = [db['net_attack_1'], db['net_attack_2'], db['net_attack_3'], db['net_attack_4'], db['net_norm']]
df_nom = ['df_net_1', 'df_net_2', 'df_net_3', 'df_net_4', 'df_net_norm']

## Nettoyage (1)

### Harmonisation noms colonnes

In [3]:
for df in df_all:
    df.columns = df.columns.str.strip()

### Ordre colonnes

In [4]:
columns_order = ['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto', 'flags', 'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst', 'modbus_response', 'label_n', 'label']
df_all = [df[columns_order] for df in df_all]

## Nettoyage (2)

### Gestion des lignes dupliquées

Nous supprimons les lignes dupliqués, et ajoutons une colonne pour informer que ce sont des lignes dupliquées.

In [5]:
for i in range(len(df_all)):
    df_all[i].loc[:, 'is_duplicate'] = df_all[i].duplicated(keep=False)

C:\Users\charl\AppData\Local\Temp\ipykernel_28068\2598439836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all[i].loc[:, 'is_duplicate'] = df_all[i].duplicated(keep=False)


In [6]:
for i in range(len(df_all)):
    df_all[i] = df_all[i].drop_duplicates()

### Modification des types

In [7]:
for i, df in enumerate(df_all):
    df['label_n'] = df['label_n'].astype('bool')
    df['label'] = df['label'].astype('category')

In [8]:
columns_to_convert = ['sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst']

for i in range(len(df_all)):
    for col in columns_to_convert:
        df_all[i][col] = df_all[i][col].astype('Int64') # ok pour les valeurs manquantes

In [9]:
for i in range(len(df_all)):
    df_all[i]['Time'] = pd.to_datetime(df_all[i]['Time'], format='mixed')

## Nettoyage (3)

### Discrétisation des ports

In [10]:
#sports_connus = set(df_all[0]['sport'].dropna().unique())
#dports_connus = set(df_all[0]['dport'].dropna().unique())

In [11]:
##def est_sport_connu(port):
#    return port in sports_connus

#def est_dport_connu(port):
#    return port in dports_connus

In [12]:
##for i in range(len(df_all)):
#    df_all[i]['est_connu_sport'] = df_all[i]['sport'].apply(est_sport_connu)
#    df_all[i]['est_connu_dport'] = df_all[i]['dport'].apply(est_dport_connu)

In [13]:
#for i in range(len(df_all)):
#    print(f"{df_nom[i]} :")
#    print(df_all[i]['est_connu_sport'].value_counts())
#    print()

In [14]:
#for i in range(len(df_all)):    
#    df_all[i] = df_all[i].drop(columns=['sport', 'dport'])

In [15]:
def get_categorie_ports(port):
    if pd.isna(port):
        return "inconnu" # remplacement des valeurs manquantes
    if port < 1024:
        return "system_ports"
    elif port <= 49151:
        return "user_ports"
    else:
        return "dynamic/private_ports"

In [16]:
for i in range(len(df_all)):
    for col in ['sport', 'dport']:
        df_all[i][col] = df_all[i][col].apply(get_categorie_ports).astype('category')

### Transformation des flags en catégories

In [17]:
for i, df in enumerate(df_all):
    df['flags'] = df['flags'].astype('category')

### One-hot de la colonne flags (faire plus tard)

In [18]:
#for i in range(len(df_all)):
#    df_all[i] = pd.get_dummies(df_all[i], columns=['flags'], prefix='flag')

In [19]:
#df_all[0].head()

Les colonnes créée ne sont pas forcément les même en fonction des datasets, car toutes les valeurs n'étaient pas présentent dans tous les datasets.  
Nous ajoutons donc des colonnes avec False si elles n'y sont pas.

#### Ajout des colonnes non créées

In [20]:
#all_columns = set()
#for df in df_all:
#    flag_columns = [col for col in df.columns if col.startswith('flag')]
#    all_columns.update(flag_columns)
#all_columns = list(all_columns)

In [21]:
#print(all_columns)

In [22]:
#for i in range(len(df_all)):
#    manquantes = set(all_columns) - set(df_all[i].columns)
#    
#    for col in manquantes:
#        df_all[i][col] = False
#    
#    df_all[i] = df_all[i].reindex(columns=list(df_all[i].columns) + [col for col in all_columns if col not in df_all[i].columns])

In [23]:
# Check si les flags avec des valeurs manquantes ont bien des False dans toutes les colonnes
#columns_to_check = ['flag_0', 'flag_10100', 'flag_10', 'flag_10000', 'flag_11000010', 'flag_11001', 'flag_11000', 'flag_1', 'flag_101001', 'flag_10010', 'flag_100', 'flag_10001']
#
#for i in range(len(df_all)):
#    df = df_all[i]
#    false_count = (~df[columns_to_check].any(axis=1)).sum()
#    print(f"{df_nom[i]} : {false_count} = False")

In [24]:
#df_all[0].head()

## Nettoyage (4)

In [25]:
for df in df_all:
    for col in ['ip_s', 'ip_d', 'modbus_fn', 'modbus_response']:
        df[col] = df[col].fillna("inconnue")
    for col in ['n_pkt_src', 'n_pkt_dst']: # La colonne redevient de type float
        df[col] = df[col].fillna(df[col].median())
    for col in ['flags']:
        df[col] = df[col].cat.add_categories('inconnue')
        df[col] = df[col].fillna('inconnue')

## Nettoyage (5)

In [26]:
for df in df_all:
    df['modbus_response'] = df['modbus_response'].str.replace(r'\[|\]', '', regex=True)
    threshold = 0.001 * len(df)
    value_counts = df['modbus_response'].value_counts()
    frequent_categories = value_counts[value_counts > threshold].index
    df['modbus_response'] = np.where(df['modbus_response'].isin(frequent_categories), df['modbus_response'], 'autre')

## Enregistrement dans PickleShare

In [27]:
data_dir = '../prep_data' 
os.makedirs(data_dir, exist_ok=True)
db = PickleShareDB(os.path.join(data_dir, 'kity'))

db['net_attack_1_clean'] = df_all[0]
db['net_attack_2_clean'] = df_all[1]
db['net_attack_3_clean'] = df_all[2]
db['net_attack_4_clean'] = df_all[3]
db['net_norm_clean'] = df_all[4]